In [1]:
try:
  import torchsummary
except:
  !pip install torchsummary

In [2]:
import numpy as np 
import pandas as pd
import os
import re
import torch
from torch import nn
from termcolor import colored 
from torchsummary import summary
from transformers import GPT2Tokenizer
from tokenizers import ByteLevelBPETokenizer
from torch.nn.utils import clip_grad_norm_
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gpt2_model/pytorch/default/1/gpt2_model (1).pth
/kaggle/input/model_gpt/pytorch/default/1/gpt_2_custom_loss_v3.pth.tar
/kaggle/input/poet-datadet-2/new_data_clean2.csv
/kaggle/input/poet-datadet-2/start_vowels.txt
/kaggle/input/poet-datadet-2/tone_dict.txt
/kaggle/input/poet-datadet-2/rhymes.txt
/kaggle/input/gpt_model_custom_loss/pytorch/default/1/gpt_2_custom_loss_v2.pth.tar


In [3]:
data = pd.read_csv('/kaggle/input/poet-datadet-2/new_data_clean2.csv')
data.shape

(255080, 2)

In [4]:
sentences = data.content.values
sentences[2]

'ai về xa mãi cô thôn\nmột mình trông khói hoàng hôn nhớ nhà\nngày em mới bước chân ra\ntuy rằng cách mặt lòng ta chưa sầu'

In [5]:
def replace_context(text):
    text = text.split('\n')
    s = ''
    for i in text:
        s = s + i + ' \\n '
    return s[:-4]

replace_context('thăm con ở trại nhi đồng\nmột ngày xuân đẹp nắng hồng thướt tha\ncon đang cùng bạn múa ca\ncành tơ phơ phất gió qua rì rào')

'thăm con ở trại nhi đồng \\n một ngày xuân đẹp nắng hồng thướt tha \\n con đang cùng bạn múa ca \\n cành tơ phơ phất gió qua rì rào'

In [6]:
sentences = [replace_context(x) for x in sentences]
sentences[0]

'thăm con ở trại nhi đồng \\n một ngày xuân đẹp nắng hồng thướt tha \\n con đang cùng bạn múa ca \\n cành tơ phơ phất gió qua rì rào'

In [7]:
def add_token(text):
    return '<s> ' + text + ' </s>'
sentences = [add_token(x) for x in sentences]
sentences[0]

'<s> thăm con ở trại nhi đồng \\n một ngày xuân đẹp nắng hồng thướt tha \\n con đang cùng bạn múa ca \\n cành tơ phơ phất gió qua rì rào </s>'

In [8]:
with open('data.txt', 'w', encoding='utf-8') as file:
    for sentence in sentences:
        file.write(sentence + '\n')

In [9]:
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = file.readlines()
    return sentences

file_path = '/kaggle/working/data.txt'

sentences = read_text_file(file_path)

for i, sentence in enumerate(sentences[:5]):
    print(f"Summary {i+1}: {sentence.strip()}")

Summary 1: <s> thăm con ở trại nhi đồng \n một ngày xuân đẹp nắng hồng thướt tha \n con đang cùng bạn múa ca \n cành tơ phơ phất gió qua rì rào </s>
Summary 2: <s> con đang cùng bạn múa ca \n cành tơ phơ phất gió qua rì rào \n tiếng ca bay lượn từng cao \n trăm con chim nhỏ ngọt ngào không gian </s>
Summary 3: <s> ai về xa mãi cô thôn \n một mình trông khói hoàng hôn nhớ nhà \n ngày em mới bước chân ra \n tuy rằng cách mặt lòng ta chưa sầu </s>
Summary 4: <s> ngày em mới bước chân ra \n tuy rằng cách mặt lòng ta chưa sầu \n nắng trôi vàng chẩy về đâu \n hôm nay mới thực bắt đầu vào thu </s>
Summary 5: <s> trời hồng chắc má em tươi \n nước trong chắc miệng em cười thêm xinh \n em đi hoài cảm một mình \n hai lòng riêng để mối tình cô đơn </s>


In [10]:
sentences = [x.strip() for x in sentences]

In [11]:
import ast
from math import ceil, floor

# try:
#     from importlib import resources
# except ImportError:
#     import importlib_resources as resources


def load_data(filename: str):

    with open(filename,'r', encoding='utf8') as file:
        text = file.read()

    content = ast.literal_eval(text)
    return content


vowels_path = "/kaggle/input/poet-datadet-2/start_vowels.txt"
start_vowels = load_data(vowels_path)

huyen = start_vowels['huyen']
sac = start_vowels['sac']
nang = start_vowels['nang']
hoi = start_vowels['hoi']
nga = start_vowels['nga']
khong_dau = start_vowels['khong_dau']

list_start_vowels = []
list_start_vowels.extend(huyen)
list_start_vowels.extend(sac)
list_start_vowels.extend(nang)
list_start_vowels.extend(hoi)
list_start_vowels.extend(nga)
list_start_vowels.extend(khong_dau)

rhyme_path = "/kaggle/input/poet-datadet-2/rhymes.txt"

rhymes_dict = load_data(rhyme_path)


even_chars = []

even_chars.extend(huyen)
even_chars.extend(khong_dau)

tone_dict = load_data("/kaggle/input/poet-datadet-2/tone_dict.txt")

In [12]:
def is_stanza(sentences: str):
    """
    Check if input is a stanza or not

    param sentences: sentences to check

    return: is stanza or not
    """
    return len(sentences.split("\n\n")) == 1


def split_word(word):
    """
        Split word by 2 part, starting and ending

        param word: word to split

        return: ending part of word
        Ex: mùa -> ùa
    """
    word_length = len(word)
    start_index = 0
    prev = ''
    for i in range(word_length):
        if prev == 'g' and word[i] == 'i':
            continue
        if prev == 'q' and word[i] == 'u':
            continue
        if word[i] in list_start_vowels:
            start_index = i
            break
        prev = word[i]
    return word[start_index:]


def compare(word1: str, word2: str):
    """
    Check 2 words rhyme if the same
    
    param word1, word2: words to check
    
    return: is the same rhyme or not
    """
    rhyme1 = split_word(word1)
    rhyme2 = split_word(word2)

    if rhyme2 in rhymes_dict[rhyme1]:
        return True
    return False


def check_rhyme_pair(prev_sentence: str, cur_sentence: str, prev_eight_words_rhyme=""):
    """
        Check 2 words rhyme if the same

        param word1, word2: words to check

        return: is the same rhyme or not
    """
    rhyme_errors = 0
    length_errors = 0

    prev_length = len(prev_sentence.split(" "))
    cur_length = len(cur_sentence.split(" "))
    s = ''

    if prev_length != 6:
        prev_sentence = "(L)" + prev_sentence
        length_errors = length_errors + 1

    if cur_length != 8:
        cur_sentence = "(L)" + cur_sentence
        length_errors = length_errors + 1

    prev_words = prev_sentence.split(" ")
    cur_words = cur_sentence.split(" ")

    if prev_eight_words_rhyme == "":
        try:
            if not compare(prev_words[5], cur_words[5]):
                cur_words[5] = cur_words[5] + "(V)"
                rhyme_errors = rhyme_errors + 1
        except Exception as e:
            s = f"{e} + {cur_sentence}"
            pass
    if prev_eight_words_rhyme != "":
        try:
            if not compare(prev_words[5], prev_eight_words_rhyme):
                prev_words[5] = prev_words[5] + "(V)"
                rhyme_errors = rhyme_errors + 1
        except Exception as e:
            s = f"{e} + {cur_sentence}"
            pass
        try:
            if not compare(prev_eight_words_rhyme, cur_words[5]):
                cur_words[5] = cur_words[5] + "(V)"
                rhyme_errors = rhyme_errors + 1
        except Exception as e:
            s = f"{e} + {cur_sentence}"
            pass
    prev_sentence =  " ".join(prev_words)
    cur_sentence =  " ".join(cur_words)

    return prev_sentence, cur_sentence, cur_words[-1], rhyme_errors, length_errors, s

def preprocess_stanza(stanza: str):
    """
    A function to process Stanza to remove all unnecessary blank

    param sentence: stanza to process

    return: stanza processed
    """
    sentences = stanza.split("\\n")
    sentences_out = []
    for sentence in sentences:
        words = sentence.split(" ")
        words_out = []
        for word in words:
            if word:
                words_out.append(word)
        sentences_out.append(" ".join(words_out))
    return "\\n".join(sentences_out)
    
def check_rhyme_stanza(stanza: str):
    """
        Check rhyme by stanza

        param stanza: input stanza to check

        return: res: stanza after check filter and error highlighted
                total_rhyme_errors: total rhyme errors
                total_length_errors: total length errors
    """
    sentences = stanza.split("\\n")
    first_words = sentences[0].split(" ")
    start_index = 0
    prev_eight_words_rhyme = ""
    total_rhyme_errors = 0
    total_length_errors = 0

    if len(first_words) == 8:
        prev_eight_words_rhyme = split_word(first_words[7])
        start_index = 1

    for i in range(start_index, len(sentences), 2):
        if i+1 == len(sentences):
            sentences.append("Missing ending sentence")
        sentences[i], sentences[i+1], prev_eight_words_rhyme, rhyme_errors, length_errors, s = check_rhyme_pair(sentences[i], sentences[i + 1], prev_eight_words_rhyme)
    
        total_rhyme_errors = total_rhyme_errors + rhyme_errors + len(s)
        total_length_errors = total_length_errors + length_errors + len(s)
    res = "\\n".join(sentences)
    return res, total_rhyme_errors, total_length_errors


def extract_consonants(word):
    # Danh sách các nguyên âm tiếng Việt
    consonants = [char for char in word if char.lower() in list_start_vowels]
    return consonants

def get_tone(word: str):
    """
        Check word is even tone or not

        param word: word to check tone

        return: even or uneven
    """
    char = split_word(word)
    chars = extract_consonants(char)
    for char in chars:
        if char not in even_chars:
            return 'uneven'
    return 'even'

def check_tone_sentence(sentence: str):
    """
        Check sentence is on the right form of even or uneven rule

        param sentence: sentence to check tone

        return: sentences after added notation to highlight error
                total_wrong_tone: total wrong tone in sentence
    """
    words = sentence.split(" ")
    length = len(words)
    if length != 6 and length != 8:
        return "(L)"+sentence, 0
    cur_tone_dict = tone_dict[length]
    total_wrong_tone = 0
    for i in cur_tone_dict:
        if get_tone(words[i]) != cur_tone_dict[i]:
            total_wrong_tone = total_wrong_tone + 1
            words[i] = words[i] + "(T)"
    return " ".join(words), total_wrong_tone

def check_tone_stanza(stanza: str):
    """
        Check stanza is on the right form of even or uneven rule

        param sentence: stanza to check tone

        return: stanza after added notation to highlight error
                total_wrong_tone: total wrong tone in sentence
    """
    sentences = stanza.split("\\n")
    total_wrong = 0
    for i in range(len(sentences)):
        current_wrong = 0
        sentences[i], current_wrong = check_tone_sentence(sentences[i])
        total_wrong = total_wrong + current_wrong
    return "\\n".join(sentences), total_wrong

def check_rule(stanza: str):
    """
    A function to check both rhyme and tone rule

    param sentence: stanza to check

    return: stanza processed
    """
    if not is_stanza(stanza):
        print(stanza + ": is not a stanza")
        return
    stanza = preprocess_stanza(stanza)
    stanza, total_rhyme_errors, total_length_errors = check_rhyme_stanza(stanza)
    stanza, total_wrong_tone = check_tone_stanza(stanza)
    return stanza, total_length_errors, total_rhyme_errors, total_wrong_tone



def calculate_score_by_error(stanza_length: int, total_length_errors=0, total_rhyme_errors=0, total_wrong_tone=0):
    """
      A function to calculate score for the Stanza by length, rhyme and tone errors
          Currently doesnt punish the length error

      param sentence: stanza_length,
                      total_length_errors,
                      total_rhyme_errors,
                      total_wrong_tone

      return: score calculated by formula that rhyme accounts for 70% score rate and 30% left for tone
    """
    try:
        num_six = ceil(stanza_length/2)
        num_eight = floor(stanza_length/2)
    
        rhyme_minus_points = 70*total_rhyme_errors/(num_six + 2*num_eight-1)
        tone_minus_points = 30*total_wrong_tone/(3*num_six+4*num_eight)
        
    
        return rhyme_minus_points + tone_minus_points + (total_length_errors * 5)
    except:
        return 50


def calculate_stanza_score(stanza: str):
   """
      A function to calculate score for the Stanza

      param sentence: stanza

      return: score  after checked by rule and calculated by formula that rhyme accounts for 70% score rate
      and 30% left for tone
   """
   stanza = preprocess_stanza(stanza)
   length = len(stanza.split("\\n"))
   stanza, total_length_errors, total_rhyme_errors, total_wrong_tone = check_rule(stanza)
   score = calculate_score_by_error(length, total_length_errors, total_rhyme_errors, total_wrong_tone)

   return score


In [13]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=["/kaggle/working/data.txt"], min_frequency=2, special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

In [14]:
tokenizer.save_model("/kaggle/working/")

['/kaggle/working/vocab.json', '/kaggle/working/merges.txt']

In [15]:
def tokenize_function(text):
    encoded = tokenizer.encode(text)
    input_ids = torch.tensor(encoded.ids).unsqueeze(0)
    attention_mask = torch.tensor(encoded.attention_mask).unsqueeze(0)
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }

In [16]:
tokenized_data = [tokenize_function(sample) for sample in sentences]

In [17]:
vocab_size = len(tokenizer.get_vocab()) 

print(colored(f"Original vocabulary size: {vocab_size}"))

Original vocabulary size: 9138


In [18]:
max_len = [len(x['input_ids'][0]) for x in tokenized_data]
max_len = max(max_len)
max_len

45

In [19]:
from torch.utils.data import DataLoader, Dataset
class TextDataset(Dataset):
  def __init__(self, tokenized_data, max_len): 
    self.data = tokenized_data
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sample = self.data[idx]
    # Pad input_ids and attention_mask to the same length (max_len)
    input_ids = sample['input_ids']
    attention_mask = sample['attention_mask']
    padded_input_ids = torch.nn.functional.pad(input_ids, (0, self.max_len - input_ids.shape[1]), value=0)
    padded_attention_mask = torch.nn.functional.pad(attention_mask, (0, self.max_len - attention_mask.shape[1]), value=0)
    return {
        'input_ids': padded_input_ids,
        'attention_mask': padded_attention_mask
    }   

In [20]:
dataset = TextDataset(tokenized_data, max_len)  
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)

In [21]:
class GPT2Model(nn.Module):
    def __init__(self, vocab_size, n_positions=1024, n_ctx=1024, n_embd=768, n_layer=12, n_head=12):
        super(GPT2Model, self).__init__()
        self.wte = nn.Embedding(vocab_size, n_embd)
        self.wpe = nn.Embedding(n_positions, n_embd)
        self.h = nn.ModuleList([nn.TransformerEncoderLayer(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, input_ids, position_ids=None):
        if position_ids is None:
            position_ids = torch.arange(input_ids.size(1), dtype=torch.long, device=input_ids.device)
            position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        inputs_embeds = self.wte(input_ids) + self.wpe(position_ids)
        hidden_states = inputs_embeds
        for block in self.h:
            hidden_states = block(hidden_states)
        hidden_states = self.ln_f(hidden_states)
        logits = self.head(hidden_states)
        return logits, hidden_states
    def get_num_parameters(self):  # Define function inside the class
        total_params = 0
        for name, param in self.named_parameters():
          if param.requires_grad:
            total_params += param.numel()
        return total_params

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpt_model = GPT2Model(vocab_size)

# gpt_model.load_state_dict(torch.load('/kaggle/input/gpt2_model/pytorch/default/1/gpt2_model (1).pth'))
gpt_model.to(device)
# gpt_model.eval()

GPT2Model(
  (wte): Embedding(9138, 768)
  (wpe): Embedding(1024, 768)
  (h): ModuleList(
    (0-11): 12 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (linear1): Linear(in_features=768, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=768, bias=True)
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=768, out_features=9138, bias=False)
)

In [23]:
import torch.optim as optim
from tqdm import tqdm

In [24]:
def save_checkpoint(state, filename= "GPT-2/gpt_2_custom_loss_v2.pth.tar"):
    print("Saving checkpoint")
    torch.save(state,filename)

def load_checkpoint(state):
    print("Load checkpoint")
    gpt_model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])

In [25]:
optimizer = optim.Adam(gpt_model.parameters(), lr=5e-5)
load_checkpoint(torch.load('/kaggle/input/model_gpt/pytorch/default/1/gpt_2_custom_loss_v3.pth.tar'))

/tmp/ipykernel_30/711693160.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_checkpoint(torch.load('/kaggle/input/model_gpt/pytorch/default/1/gpt_2_custom_loss_v3.pt

Load checkpoint


In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7, top_k=50, device='cuda'):

    model.eval()  # Set model to evaluation mode
    
    # Tokenize the prompt
    input_ids = torch.tensor(tokenizer.encode(prompt).ids)
    input_ids = input_ids.unsqueeze(0) 
    input_ids = input_ids.to(device)
    
    # Create attention mask
    # attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)
    generated = input_ids[0].tolist()  # Include the prompt tokens
    current_length = len(generated)
    
    with torch.no_grad():
        while current_length < max_length:
            # Get model output
            outputs = model(input_ids)
            next_token_logits = outputs[0][0, -1, :] / temperature  # Take last token prediction
            
            # Apply top-k filtering
            if top_k > 0:
                top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
                top_k_tokens = [tokenizer.decode([idx.item()]) for idx in top_k_indices]
                next_token_logits = torch.full_like(next_token_logits, float('-inf'))
                next_token_logits[top_k_indices] = top_k_logits
            
            # Sample from the filtered distribution
            probs = F.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            # Append the next token to input_ids
            input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)
            generated.append(next_token.item())
            current_length += 1
            
            # Try to decode the current sequence
            try:
                current_text = tokenizer.decode(generated)
                # If we get a natural break point (e.g., end of sentence), we can stop
                if current_text.endswith(('.', '!', '?', '\n')) and current_length > len(encoded.ids):
                    break
            except:
                continue
    
    try:
        generated_text = tokenizer.decode(generated)
    except:
        # If decoding fails, return what we have up to the last successful token
        generated_text = tokenizer.decode(generated[:-1])
    
    return generated_text


prompt = "<s> thăm con ở trại nhi đồng" 
generated_text = generate_text(
    model=gpt_model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=max_len,
    temperature=0.7,
    top_k=50,
    device=device
)

print(f"Prompt: {prompt}")
print(f"Generated text: {generated_text}")


Prompt: <s> thăm con ở trại nhi đồng
Generated text:  thăm con ở trại nhi đồng \n để rồi ta vẫn đứng ngồi không màn \n còn đây cũng thật đáng yêu \n em đây là để nói lời yêu thương 


In [29]:
prompt = "<s> mùa xuân" 
generated_text = generate_text(
    model=gpt_model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=max_len,
    temperature=0.7,
    top_k=50,
    device=device
)

print(f"Prompt: {prompt}")
print(f"Generated text: {generated_text}")

Prompt: <s> mùa xuân
Generated text:  mùa xuân về chốn hư hao \n để anh em chẳng thể nào dám gần \n nhớ sao duyên dáng em ơi \n để ta ôm ấp cho mình vì con 


In [48]:
import random

lst_prompts = []
for sentence in sentences[:100]:
    random_number = random.randint(1, 6)
    a = sentence.split(' ')
    s = ''
    for char in a[:random_number+1]:
        s = s + char + ' '
    lst_prompts.append(s[:-1])
    

In [62]:
gen_sentences = []
for prompt in lst_prompts:
    generated_text = generate_text(
        model=gpt_model,
        tokenizer=tokenizer,
        prompt=prompt,
        max_length=max_len,
        temperature=0.8,
        top_k=20,
        device=device
        )
    gen_sentences.append(generated_text)

In [32]:
!pip install einops

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
from einops import rearrange
import math

In [34]:
class ScaleDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        batch_size, head, length, d_tensor = k.size()

        score = torch.einsum("bhid,bhjd->bhij",q,k)
        score = score/math.sqrt(d_tensor)

        if mask is not None:
            score = score.masked_fill(mask == 0, -e)

        score = self.softmax(score)

        v = score @ v

        return v, score

In [35]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, d_model*n_head)
        self.w_k = nn.Linear(d_model, d_model*n_head)
        self.w_v = nn.Linear(d_model, d_model*n_head)
        self.w_concat = nn.Linear(d_model*n_head, d_model)

    def forward(self, x, mask=None):
        q, k, v = self.w_q(x), self.w_k(x), self.w_v(x)

        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.n_head), (q, k, v))

        out, attention = self.attention(q, k, v, mask=mask)

        # 4. concat and pass to linear layer
        # out = self.concat(out)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.w_concat(out)

        return out

In [36]:
class SelfAttentionLstm(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers,n_head):
        super(SelfAttentionLstm, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.multi_attention = MultiHeadAttention(d_model=input_size,n_head=4)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x = self.multi_attention(x)
         
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to("cuda")
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to('cuda')

        out, _ = self.lstm(x, (h0, c0))

        out = out[: ,-1, : ]
        return out

In [37]:
def split_list_with_indices(input_list):
    # Loại bỏ số 0 ở đầu list
    start_idx = 0
    while start_idx < len(input_list) and input_list[start_idx] == 0:
        start_idx += 1
        
    # Loại bỏ số 0 ở cuối list
    end_idx = len(input_list) - 1
    while end_idx >= 0 and input_list[end_idx] == 0:
        end_idx -= 1
        
    # Tách list và lưu indices
    result = []
    indices = []
    current_start = start_idx
    
    i = start_idx
    while i <= end_idx:
        if input_list[i] == 266 or i == end_idx:
            end = i + 1 if input_list[i] == 266 else i + 1
            sublist = input_list[current_start:end]
            result.append(sublist)
            indices.append((current_start, end - 1))
            current_start = end
            i = end
        else:
            i += 1
            
    return indices


input_list = [0, 1027, 378, 640, 3696, 2255, 615, 266, 82, 519, 419, 891, 1021, 371, 581, 2617, 2014, 266, 82, 544, 449, 391, 539, 823, 510, 266, 82, 371, 1353, 445, 1236, 492, 1184, 1837, 528, 225, 2, 0, 0, 0, 0, 0, 0, 0, 0]


indices = split_list_with_indices(input_list.copy())
indices


[(1, 7), (8, 17), (18, 25), (26, 36)]

In [38]:
def get_idx_two_line(lm_logits):
    token = torch.argmax(lm_logits, dim= 2)
    token = token[0].tolist()
    return split_list_with_indices(token.copy())

    
def loss_kho_tho(lm_logits,embedding):
    lm_logits = torch.unsqueeze(lm_logits,0)
    pair_list = get_idx_two_line(lm_logits)
    embedding = torch.unsqueeze(embedding,0)
    
    total_lost = 0
    loss = nn.MSELoss().to(device)
    for i in range(len(pair_list)-1):
        one = pair_list[i]
        two = pair_list[i+1]

        if one == None or two == None:
          continue
        
        # Kiểm tra shape của embedding trước khi đưa vào LSTM
        embedd_slice_one = embedding[:,one[0]:one[1],:]
        embedd_slice_two = embedding[:,two[0]:two[1],:]
        if embedd_slice_one.size(1) == 0 or embedd_slice_two.size(1) == 0:
            continue
            
        embedd_one = head_gpt(embedd_slice_one)
        embedd_two = head_gpt(embedd_slice_two)

        total_lost += loss(embedd_one,embedd_two)
    return total_lost     
     

In [39]:

def train_model(model, dataloader, optimizer,tokenizer, device, epochs=3):
    model.train()
    losses = []
    for epoch in range(epochs):
        with tqdm(total=len(dataloader), desc=f"Epoch {epoch}") as progress_bar:
          for batch in dataloader:
            input_ids = batch['input_ids'].squeeze(1).to(device)            
            
            outputs = model(input_ids)[0]
            embedding = model.wte(input_ids)
              
            shift_logits = outputs[..., :-1, :].contiguous()
            shift_labels = input_ids[..., 1:].contiguous()
            loss_fct = nn.CrossEntropyLoss()
              
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            loss = loss + sum([loss_kho_tho(shift_logits[i],embedding[i]) for i in range(shift_logits.shape[0])])*100
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
              
            optimizer.step()
            losses.append(loss.item())
            
            progress_bar.set_postfix({'loss': loss.item()})  # Set 'loss' key-value pair
            progress_bar.update(1)
        print(f"Epoch: {epoch}, Average Loss: {sum(losses) / len(losses)}")
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint, filename= "/kaggle/working/gpt_2_custom_loss_v2.pth.tar")
    return losses

In [42]:
# import torch.optim as optim
# from tqdm import tqdm

# head_gpt = SelfAttentionLstm(input_size=768,hidden_size=768, num_layers=2,n_head=4).to('cuda')
# gpt_losses = train_model(gpt_model, dataloader, optimizer, tokenizer, device, epochs=2)